In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('VideoMatchingSrate.csv', sep='\t')

In [3]:
df.head()

FilePath, VideoName, SRate, Start, Stop, Duration
0  /Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK/...
1  /Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK/...
2  /Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ/...
3  /Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ/...
4  /Volumes/methlab_data/HBN/EEG-ET/NDARAA306NT2/...

In [4]:
cols = df.iloc[:,0].apply(lambda s: s.split(','))

In [5]:
cols

0       [/Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK...
1       [/Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK...
2       [/Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ...
3       [/Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ...
4       [/Volumes/methlab_data/HBN/EEG-ET/NDARAA306NT2...
                              ...                        
8285    [/Volumes/methlab_data/HBN/EEG-ET/NDARZZ830JM7...
8286    [/Volumes/methlab_data/HBN/EEG-ET/NDARZZ830JM7...
8287    [/Volumes/methlab_data/HBN/EEG-ET/NDARZZ993CEV...
8288    [/Volumes/methlab_data/HBN/EEG-ET/NDARZZ993CEV...
8289    [/Volumes/methlab_data/HBN/EEG-ET/NDARZZ993CEV...
Name: FilePath, VideoName, SRate, Start, Stop, Duration, Length: 8290, dtype: object

In [6]:
for index, value in cols.items():
    if len(value)>6:
        print('indice:', index)
        print('value:', value)
        print('-')
        print('valores extra:', value[0], value[1])
        print('inicio,fin,dur (ANTERIOR):', cols[index-1][-3],cols[index-1][-2],cols[index-1][-1])
        print('inicio,fin,dur (ACTUAL):', value[-3], value[-2], value[-1])
        print('-')
        print(int(cols[index-1][-1])-int(cols[index-1][-2]))
        print('-')
        print('fin-start(anterior):',int(cols[index-1][-2])-int(cols[index-1][-3]))
        print('------------------------')

indice: 217
value: ['2.582658e+08', ' 1.705471e+08', ' /Volumes/methlab_data/HBN/EEG-ET/NDARAP782TVC/NDARAP782TVC_Video2_Samples_ET.mat', ' Fractals', ' 120', ' 2608780758', ' 2771767670', ' 162986912']
-
valores extra: 2.582658e+08  1.705471e+08
inicio,fin,dur (ANTERIOR):  2303153653  2390872282  2561419422
inicio,fin,dur (ACTUAL):  2608780758  2771767670  162986912
-
170547140
-
fin-start(anterior): 87718629
------------------------
indice: 2339
value: ['2.510055e+08', ' 1.705469e+08', ' /Volumes/methlab_data/HBN/EEG-ET/NDARGJ627BL2/NDARGJ627BL2_Video2_Samples_ET.mat', ' Fractals', ' 120', ' 3514859560', ' 3677846974', ' 162987414']
-
valores extra: 2.510055e+08  1.705469e+08
inicio,fin,dur (ANTERIOR):  3097963641  3178422267  3348969140
inicio,fin,dur (ACTUAL):  3514859560  3677846974  162987414
-
170546873
-
fin-start(anterior): 80458626
------------------------
indice: 2384
value: ['4.159950e+08', ' 1.705565e+08', ' /Volumes/methlab_data/HBN/EEG-ET/NDARGL359WZA/NDARGL359WZA_Video2

- En las filas que tienen columnas de mas
    - Hay que sacar los dos primeros valores y agregarlos a la fila anterior
    - El primer valor extra parece ser un tiempo que no sabemos a que corresponde
    - El segundo valor extra sería la duracion
    - De la fila anterior el ultimo es stop y el anteultimo es el start.
    - El antepenultimo no sabemos que es
    - El indice correspondiente al archivo '/Volumes/methlab_data/HBN/EEG-ET/NDARML926NEG/NDARML926NEG_Video2_Samples_ET.mat' (en mi caso el 4299-4300) tiene mas entradas erroneas que el resto

In [7]:
filas = []
colnames =  ['FilePath', 'VideoName', 'SRate', 'Start', 'Stop', 'Duration']
for index, value in cols.items():
    if len(value)>6:
        fila_ant_d = filas.pop(-1)
        fila_ant_corr_val = list(fila_ant_d.values())[:3] + [list(fila_ant_d.values())[4]] + [value[1]]
        fila_ant_corr = dict(zip(colnames, fila_ant_corr_val))
        filas.append(fila_ant_corr)
        fila = dict(zip(colnames, value[2:]))
    else:
        fila = dict(zip(colnames, value))
    filas.append(fila)
df_filas_corr = pd.DataFrame(filas)

In [8]:
df_filas_corr.head()

FilePath    VideoName SRate  \
0  /Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK/...        Diary    60   
1  /Volumes/methlab_data/HBN/EEG-ET/NDARAA075AMK/...   Despicable    60   
2  /Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ/...        Diary    60   
3  /Volumes/methlab_data/HBN/EEG-ET/NDARAA117NEJ/...   Despicable    60   
4  /Volumes/methlab_data/HBN/EEG-ET/NDARAA306NT2/...   Despicable    60   

         Start         Stop    Duration  
0   2887191440   3004584578   117393138  
1   3788400246   3958945796   170545550  
2   3333897971   3451293101   117395130  
3   4710198376   4880739124   170540748  
4   3125258026   3295795728   170537702

In [9]:
videos = df_filas_corr.VideoName.unique()
videos

array([' Diary', ' Despicable', ' Present', ' Fractals', ' 162988064'],
      dtype=object)

In [10]:
df_filas_corr.VideoName.value_counts()

 Diary         2558
 Despicable    2555
 Present       1807
 Fractals      1369
 162988064        1
Name: VideoName, dtype: int64

In [11]:
df_filas_corr[df_filas_corr.VideoName == videos[-1]]

FilePath   VideoName  \
4300   286826838   162988064   

                                                  SRate        Start  Stop  \
4300   /Volumes/methlab_data/HBN/EEG-ET/NDARML926NEG...   Despicable   120   

         Duration  
4300   5605429092

In [12]:
fant = cols.iloc[4299]
fact = cols.iloc[4300]

In [13]:
fant

['/Volumes/methlab_data/HBN/EEG-ET/NDARML926NEG/NDARML926NEG_Video1_Samples_ET.mat',
 ' Fractals',
 ' 120',
 ' 4331406439',
 ' 4536394842',
 ' 4660233616']

In [14]:
fact

['4.823222e+09',
 ' 4.918152e+08',
 ' 286826838',
 ' 162988064',
 ' /Volumes/methlab_data/HBN/EEG-ET/NDARML926NEG/NDARML926NEG_Video2_Samples_ET.mat',
 ' Despicable',
 ' 120',
 ' 5605429092',
 ' 5775976013',
 ' 170546921']

In [15]:
int(fant[-1]) - int(fant[-2])

123838774

In [16]:
int(fant[-2]) - int(fant[-3])

204988403

In [17]:
float(fact[0]) - int(fant[-1])

162988384.0

In [18]:
float(fact[1]) - float(fact[0])

-4331406800.0

In [19]:
int(float(fact[0]) - float(fant[-1])) == int(fact[3])

False

In [20]:
int(float(fact[0]) - float(fant[-1])) - int(fact[3])

320

In [21]:
# Nueva corrreccion incluyendo la fila especifica esta
# parece ser que el ultimo es el start, el primero de mas es el stop y el cuarto es la dur
filas = []
colnames =  ['FilePath', 'VideoName', 'SRate', 'Start', 'Stop', 'Duration']
for index, value in cols.items():
    if len(value)>6:
        # saco la fila anterior que estaba mal
        fila_ant_d = filas.pop(-1)
        if index==4300:
            fila_ant_corr_val = list(fila_ant_d.values())[:3] + [list(fila_ant_d.values())[-1]] + [value[0]]
            # nueva fila para agregar
            fila = dict(zip(colnames, value[4:]))
        else:
            fila_ant_corr_val = list(fila_ant_d.values())[:3] + [list(fila_ant_d.values())[4]] + [value[1]]
            # nueva fila para agregar
            fila = dict(zip(colnames, value[2:]))
        fila_ant_corr = dict(zip(colnames, fila_ant_corr_val))
        # agrego la fila anterior corregida
        filas.append(fila_ant_corr)
    else:
        fila = dict(zip(colnames, value))
    filas.append(fila)
df_filas_corr = pd.DataFrame(filas)

In [22]:
# algunos chequeos basicos
df_filas_corr.VideoName.unique()

array([' Diary', ' Despicable', ' Present', ' Fractals'], dtype=object)

In [23]:
df_filas_corr.SRate.unique()

array([' 60', ' 120', ' 30'], dtype=object)

In [24]:
# chequeo duracion de un video
vid_id = 0
vid_mean = df_filas_corr[df_filas_corr.VideoName==videos[vid_id]].Duration.astype(int).mean()
print('mean:', vid_mean)
vid_std = df_filas_corr[df_filas_corr.VideoName==videos[vid_id]].Duration.astype(int).std()
print('std:', vid_std)

mean: 117252347.65011728
std: 4023325.607407642


In [25]:
# lo guardo
df_filas_corr.to_csv('VideoMatchingSrate_corr.csv', index = False)